# Surface age prediction using saved models

This code predict the SW age at 1 AU using the already saved models.

For itokawa, models saved in "Itokawa_Models" (These models were saved using the "Itokawa_model_training.ipynb")

For Eros, models saved in "Eros_Models" (These models were saved using the "Eros_model_training.ipynb").

In "Surface age correction", assign "a" 1.3 for Itokawa and 1.4 for Eros (Semi-major axis of Itokawa and Eros).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import numpy as np
import pandas as pd
import pickle
import sklearn
import joblib
import os
import math

In [ ]:
pip install ml_dtypes>=0.5

In [ ]:
pip install -U jax jaxlib ml_dtypes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 MB 9.5 MB/s eta 0:00:00
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.7.2
    Uninstalling jaxlib-0.7.2:
      Successfully uninstalled jaxlib-0.7.2
  Attempting uninstall: jax
    Found existing installation: jax 0.7.2
    Uninstalling jax-0.7.2:
      Successfully uninstalled jax-0.7.2


In [ ]:
pip install scikeras

In [ ]:
from scipy.interpolate import interp1d
from scipy.ndimage import gaussian_filter1d
from keras.layers import Dense, Conv1D, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from tensorflow.keras.models import Sequential
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import VotingRegressor,GradientBoostingRegressor,RandomForestRegressor, ExtraTreesRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.base import RegressorMixin, BaseEstimator
from sklearn.neighbors import KNeighborsRegressor
from scikeras.wrappers import KerasRegressor
from sklearn.metrics import r2_score
from sklearn.utils.validation import check_is_fitted
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model

In [ ]:
#CNN model
def create_cnn_model_1(input_shape,learning_rate=0.001):
    model = Sequential()
    model.add(Conv1D(filters=32, kernel_size=3, activation="relu", padding="same", input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=16, kernel_size=2, activation="relu", padding="same"))
    model.add(BatchNormalization())

    model.add(Flatten())
    model.add(Dense(1, activation="relu"))
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=["mean_absolute_error"])
    return model

In [ ]:
#Custom Keras Regressor class for CNN-based regression tasks.

class CNNRegressor(KerasRegressor):
    _estimator_type = "regressor"

    def __init__(self, input_shape, learning_rate=0.001, epochs=10, batch_size=32, verbose=0, **kwargs):
        super().__init__(**kwargs)
        self.input_shape = input_shape
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose
        #super().__init__(build_fn=self._keras_build_fn, **kwargs)

    def _keras_build_fn(self):
        return create_cnn_model_1(self.input_shape, self.learning_rate)

In [ ]:
# Define choices for H ion irradiation (H) and lasser irradiation (L)
#1-olivine with H+, 2-pyroxene with H+, 3- OC/mixtures with H+
# 10- olivine with laser, 11- Pyroxene with laser, 12- OC/mixture with laser

choices = {'H': 2, 'L': 11}

# Load the input data sheet
input_file = "/Itokawa_model.xlsx"  # Replace with your file name
input_data = pd.read_excel(input_file)

# Retain the first two columns as identifiers
identifiers = input_data.iloc[:, :3]  # First two columns
corrected = identifiers.copy()

# Separate features (X) and target (y), excluding the first two columns
X = input_data.iloc[:, 5:-1].to_numpy()  # Columns 3 to second-last as features
y = input_data.iloc[:, -1].copy()       # The last column (target)

# Initialize a DataFrame to store predictions for both H and L
all_predictions_combined = identifiers.copy()  # Start with identifiers

print("Shape of X:", X.shape)
print(input_data.columns[1:])

Shape of X: (928, 9)
Index(['550nm', '594nm', '638nm', '682nm', '726nm', '770nm', '814nm', '858nm',
       '902nm'],
      dtype='object')


In [ ]:
output_folder = "Predictions"

for choice, adjustment_value in choices.items():

  y_adjusted  = y + adjustment_value

  X_modified = np.hstack([X, y_adjusted.values.reshape(-1, 1)])

print(X_modified.shape)

(928, 10)


In [ ]:
# --- Paths and parameters ---
model_folder = "/Itokawa_Models"
n_iterations = 30

# Ensemble weights (same as before)
weights = np.array([4, 4, 1, 5, 1])
weights = weights / weights.sum()

# Placeholder for all iterations' predictions
all_preds = []

for choice, adjustment_value in choices.items():
  # Adjust the target values based on the current choice
  y_adjusted  = y + adjustment_value

   # Reconstruct the modified dataset for predictions
  X_modified = np.hstack([X, y_adjusted.values.reshape(-1, 1)])

  for iteration in range(1, 31):
      print(f"Loading ensemble {iteration}")

      # Load models
      cnn_path = os.path.join(model_folder, f"Model_{iteration}_cnn.keras")
      gbr_path = os.path.join(model_folder, f"Model_{iteration}_gbr.joblib")
      knn_path = os.path.join(model_folder, f"Model_{iteration}_knn.joblib")
      etr_path = os.path.join(model_folder, f"Model_{iteration}_etr.joblib")
      rft_path = os.path.join(model_folder, f"Model_{iteration}_rft.joblib")

      for path in [cnn_path, gbr_path, knn_path, etr_path, rft_path]:
          if not os.path.exists(path):
              print(f"Missing file for iteration {iteration}: {os.path.basename(path)}")
              continue

      # Load models
      cnn_reg = load_model(cnn_path)
      gbr = joblib.load(gbr_path)
      knn = joblib.load(knn_path)
      etr = joblib.load(etr_path)
      rft = joblib.load(rft_path)

      preds = [
          cnn_reg.predict(X_modified),
          gbr.predict(X_modified),
          knn.predict(X_modified),
          etr.predict(X_modified),
          rft.predict(X_modified)
      ]

      # Weighted average
      preds = [np.ravel(p) for p in preds]
      weighted_pred = np.average(preds, axis=0, weights=weights)

      y_pred_scaled = 10**weighted_pred - 1

      all_preds.append(y_pred_scaled)
      all_predictions_combined[f"{iteration}_{choice}"] = y_pred_scaled

  predictions_array = np.array(all_preds)

  mean_predictions = predictions_array.mean(axis=0)
  std_predictions = predictions_array.std(axis=0)

  all_predictions_combined[f"Mean_{choice}"] = mean_predictions
  all_predictions_combined[f"Std_{choice}"] = std_predictions

  corrected[f"Mean_{choice}"] = mean_predictions
  corrected[f"Std_{choice}"] = std_predictions

# Save all predictions and statistics to a single Excel file
output_file = "/Predictions/Itokawa_results_H_and_L.xlsx"
all_predictions_combined.to_excel(output_file, index=False)

print(f"Predictions and statistics for both 'H' and 'L' saved to {output_file}")


Loading ensemble 1
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
Loading ensemble 2
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
Loading ensemble 3
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
Loading ensemble 4
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
Loading ensemble 5
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
Loading ensemble 6
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
Loading ensemble 7
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
Loading ensemble 8
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
Loading ensemble 9
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
Loading ensemble 10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Loading ensemble 11
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Loading ensemble 12
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
Loading ensemble 13
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
Loading ensemble 14
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Loading ensemble 15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Loading ensemble 16
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
Loading ensemble 17
29/29 ━━━━━━━━━━━

Surface age correction


In [ ]:
#surface age corrected (H ion) = surface age at 1 AU * (distance)^2

#For Itokawa  a = 1.3 for Eros a = 1.4
a=1.3

H_corrected = merged_df['Mean_H']*a**2
H_std_corrected = merged_df['Std_H']*a**2

merged_df['H_corrected'] = H_corrected
merged_df['H_std_corrected'] = H_std_corrected

In [ ]:
print(H_corrected)

In [ ]:
#base on Grun et. al (1991) and Divine et. al (1993) and Jehn et. al (200)

W = 10**(-12)   #weight of the particle in g
B = (math.log10(W)+11.5)/5.5
B2 = 1-B**2
gamma = (math.log10(W)+12)/6
F = 10**(-4)    # flux at 1 AU
T = 365*24*3600 #time in s
A1AU = 0.00035478

o6 = (0.138+0.142 * am+ 0.408*a**2)** - 1
o12 =(6.8-1.96 * a + 0.16 * (a**2))
frel_1 = (5**B2)*o6
frel_2 = gamma**2+(1-gamma**2)*o12
frel = frel_1/frel_2
f = F*frel  #fluence
v = 11300*a**2 - 38900*a + 42600  #velocity m/s interpotalet between values at 1 Au, 2 Au and 3 AU.

A= 0.5*W/1000*(v**2)*f*T  #1/2*particle weight*Impact average velocity*Flux*Time per year in seconds

#surface age corrected (laser) = surface age at 1 AU * A1AU/A

L_corrected = all_predictions_combined["Mean_L"]*(A1AU/A)
L_std_corrected =all_predictions_combined['Std_L']*(A1AU/A)

all_predictions_combined['L_corrected'] = L_corrected
all_predictions_combined['L_std_corrected'] = L_std_corrected

corrected['L_corrected'] = L_corrected
corrected['L_std_corrected'] = L_std_corrected


# Save to an Excel file
merged_df.to_excel('/Surface_age_Itokawa.xlsx', index=False)